# Solution Part II.
This section will address the problems raised in the previous notebook. The objective is to create "aggregated features" for each user, based on their financial reports, and use them to train the models.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.preprocessing import OneHotEncoder

In [2]:
users = pd.read_csv('users.csv')
credits = pd.read_csv('credit_reports.csv')
df = users.merge(credits, left_on='id', right_on='user_id')

## 3. Data Preparation

In [3]:
df.dtypes

id                                      int64
monthly_income                          int64
monthly_outcome                         int64
class                                   int64
user_id                                 int64
institution                            object
account_type                           object
credit_type                            object
total_credit_payments                 float64
payment_frequency                      object
amount_to_pay_next_payment            float64
account_opening_date                   object
account_closing_date                   object
maximum_credit_amount                 float64
current_balance                       float64
credit_limit                          float64
past_due_balance                      float64
number_of_payments_due                float64
worst_delinquency                     float64
worst_delinquency_date                 object
worst_delinquency_past_due_balance    float64
dtype: object

#### Similar Classes

In [4]:
df['institution'].value_counts()

BANCO                                       4285
COMUNICACIONES                              1680
MERCANCIA PARA HOGAR Y OFICINA              1407
SOCIEDAD FINANCIERA DE OBJETO MULTIPLE      1298
FINANCIERA                                  1056
TIENDA DEPARTAMENTAL                         972
BANCOS                                       738
TIENDA COMERCIAL                             576
MICROFINANCIERA                              540
SERVICIOS                                    501
AUTOMOTRIZ                                   311
MERCANCIA PARA LA CONSTRUCCION               253
SERVICIO DE TELEVISION DE PAGA               253
ADMINISTRADORAS DE CARTERA                   225
TELEFONIA CELULAR                            225
GOBIERNO                                     201
GUBERNAMENTALES                              194
CIA Q  OTORGA                                169
COBRANZA                                     160
SOFOL PRESTAMO PERSONAL                      157
TELEFONIA LOCAL Y DE

There are a couple of classes that could be joined together, such as "KONFIO-10222", "KONFIO-10319" and "KONFIO", or "BANCO" and "BANCOS". We'll clean such classes to reduce the dimensionality of the dataset.

In [5]:
classes = {
    "BANCOS": "BANCO",
    "GUBERNAMENTALES": "GOBIERNO",
    "TELEFONIA CELULAR": "COMUNICACIONES",
    "TELEFONIA LOCAL Y DE LARGA DISTANCIA": "COMUNICACIONES",
    "TELCELMETRO": "COMUNICACIONES",
    "SERVICIO DE TELEVISION DE PAGA": "SERVICIOS",
    "SERVS. GRALES.": "SERVICIOS",
    "CABLEVISION DF": "SERVICIOS",
    "COMPANIA DE FINANCIAMIENTO AUTOMOTRIZ": "AUTOMOTRIZ",
    "SOCIEDAD FINANCIERA DE OBJETO MULTIPLE": "FINANCIERA",
    "COMPANIA DE FINANCIAMIENTO DE MOTOCICLET": "AUTOMOTRIZ",
    "TIENDA DE AUTOSERVICIO": "TIENDAS",
    "VENTA POR CATALOGO": "TIENDAS",
    "TIENDA DE ROPA": "TIENDAS",
    "TIENDA COMERCIAL": "TIENDAS",
    "TIENDA DEPARTAMENTAL": "TIENDAS",
    "MERCANCIA PARA HOGAR Y OFICINA": "TIENDAS",
    "SOCIEDADES FINANCIERAS POPULARES": "FINANCIERA",
    "MICROFINANCIERA": "FINANCIERA",
    "MERCANCIA PARA LA CONSTRUCCION": "BIENES RAICES",
    "HIPOTECAGOBIERNO": "BIENES RAICES",
    "HIPOTECARIA": "BIENES RAICES",
    "HIPOTECARIO NO BANCARIO": "BIENES RAICES",
    "FONDOS Y FIDEICO": "FONDOS Y FIDEICOMISOS",
    "FACTORAJE": "FONDOS Y FIDEICOMISOS",
    "ADMINISTRADORAS DE CARTERA": "FONDOS Y FIDEICOMISOS",
    "FONDOS Y FIDEIC": "FONDOS Y FIDEICOMISOS",
    "ARRENDADORA": "ARRENDAMIENTO"
}

print(f"Before: {len(df['institution'].unique())}")

df['institution'] = df['institution'].replace(classes)
df['institution'] = df['institution'].str.replace(r'KONFIO.*', 'KONFIO', regex=True)

print(f"After: {len(df['institution'].unique())}")

Before: 54
After: 25


### Dates

In [6]:
def date_formatter(date_series: pd.Series) -> pd.Series:
    dates = date_series.str.split('/', expand=True)
    dates.columns = ['month', 'day', 'year']
    dates['year_float'] = dates['year'].astype(float)
    
    dates['year'] = dates['year'].where((dates['year_float'] >= 20),
                                        ('20' + dates['year']))
    
    dates['year'] = dates['year'].where((dates['year_float'] <= 20),
                                        ('19' + dates['year']))
    
    return pd.to_datetime(dates['year']
                          + '-' + dates['month']
                          + '-' + dates['day'],
                          format='%Y-%m-%d')

In [7]:
for column in df.columns[df.dtypes.index.str.contains('date')]:
    df[column] = date_formatter(df[column])

In [8]:
df[df.columns[df.dtypes.index.str.contains('date')]].head()

,account_opening_date,account_closing_date,worst_delinquency_date
0,2014-11-10,2015-12-05,NaT
1,2015-12-05,2016-10-14,2016-05-28
2,2016-09-02,NaT,2017-03-03
3,2016-10-14,NaT,2017-03-03
4,2016-12-27,2017-06-27,NaT


### Dropping Missing Values

In [9]:
df = df.dropna(subset=df.columns[(df.isna().sum()<100) & (df.isna().sum()>=0)])

## Delinquency
In order to represent how big of a deal is a missing payment for all credits, we'll represent the data `payment_frequency` in days so all data is homogeneous.

In [10]:
df['payment_frequency'].value_counts()

Mensual                                 11143
Semanal                                  2729
Quincenal                                 998
Una sola exhibición                       875
Catorcenal                                293
Pago mínimo para cuentas revolventes       98
Anual                                      76
Bimestral                                  30
Trimestral                                  9
Deducción del salario                       1
Name: payment_frequency, dtype: int64

In [11]:
frequency_days = {
        "Mensual": 365.25/12,
        "Semanal": 7,
        "Quincenal": 15,
        "Una sola exhibición": 1,
        "Catorcenal": 14,
        "Anual": 365.25,
        "Bimestral": 365.25/6,
        "Trimestral": 365.25/4,
        "Pago mínimo para cuentas revolventes": 365.25*60
}

df['pmt_freq_days'] = df['payment_frequency'].map(frequency_days).copy()

I chose `365*60` for the minimum payment, since in Argentina, a debt might be collected in a 5 year window since the last attempt to locate the borrower. After that period, the borrower is not legally obligated to make the payment.

Depending on the kind of data that `Pago mínimo para cuentas revolventes` has, this decision might be detrimental. The other option being using a Zero instead. For that reason, I'm creating a second variable called `pmt_freq_days_0` in order to compare them.

In [12]:
frequency_days["Pago mínimo para cuentas revolventes"] = 0

df['pmt_freq_days_0'] = df['payment_frequency'].map(frequency_days).copy()

In [13]:
df['delinquency'] = df['worst_delinquency'] * df['pmt_freq_days']
df['delinquency_0'] = df['worst_delinquency'] * df['pmt_freq_days_0']

## Current and previous loans
In order to extract features, I'll define a function that counts how many loans a person currently has and how many they had in the previous 2, 3, 6, 12, 18 and 24 months.
This function will also sum the total debt for each period as well as their respective delinquency values.

In [14]:
max(df['account_closing_date'].max(), df['account_opening_date'].max())

Timestamp('2018-03-03 00:00:00')

We'll measure the loan portfolio at march 3rd, 2018, which is the latest date that we have in the dataset.

In [46]:
from datetime import datetime

def feature_generator(dataf):
    today = datetime(2018, 3, 3)
    user_df = dataf.copy()
    user_df['account_closing_date'] = user_df['account_closing_date'].fillna(today)
    
    current = today - timedelta(days=30)
    two = today - timedelta(days=60)
    three = today - timedelta(days=90)
    six = today - timedelta(days=180)
    twelve = today - timedelta(days=365.25)
    eighteen = today - timedelta(days=365.25*1.5)
    twentyfour = today - timedelta(days=365.25*2)
    sixty = today - timedelta(days=365.25*5)
    
    intervals = [today, current, two, three, six, twelve, eighteen, twentyfour, sixty]
    
    result = user_df.loc[:,['user_id', 'monthly_income', 'monthly_outcome', 'class']].head(1)
    
    # features per interval
    for i, inter in enumerate(intervals):
        if i+1 < len(intervals):
            next_inter = intervals[i+1]
            
            mask1 = (user_df['account_opening_date']>=next_inter) & (user_df['account_opening_date']<inter)
            mask2 = (user_df['account_opening_date']<next_inter) & (user_df['account_closing_date']>=next_inter)
            
            data = user_df.loc[mask1 | mask2]
            result[f'active_loans_interval_{i}'] = data.shape[0]
            result[f'sum_loans_interval_{i}'] = data['current_balance'].sum()
            
            mask3 = (user_df['worst_delinquency_date']>=next_inter) & (user_df['worst_delinquency_date']<inter)
            data = user_df.loc[mask3]
            result[f'delinquencies_interval_{i}'] = data.shape[0]
            result[f'delinq_sum_interval_{i}'] = (data['delinquency']*data['worst_delinquency_past_due_balance']).sum()
            
    # More features
    result['tenure'] = (today - user_df['account_opening_date'].min()).days
    result['longest_duration'] = user_df['total_credit_payments'].max()
    result['preferred_institution'] = user_df['institution'].value_counts().index[0]
    result['preferred_account_type'] = user_df['account_type'].value_counts().index[0]
    result['preferred_credit_type'] = user_df['credit_type'].value_counts().index[0]
    result['preferred_payment_freq'] = user_df['payment_frequency'].value_counts().index[0]
    
    return result.iloc[0]

In [47]:
test = df.loc[df['user_id']==0]

feature_generator(test)

user_id                                    0
monthly_income                        255359
monthly_outcome                       151439
class                                      1
active_loans_interval_0                   21
sum_loans_interval_0             1.20224e+06
delinquencies_interval_0                   0
delinq_sum_interval_0                      0
active_loans_interval_1                   21
sum_loans_interval_1             1.20224e+06
delinquencies_interval_1                   0
delinq_sum_interval_1                      0
active_loans_interval_2                   21
sum_loans_interval_2             1.20224e+06
delinquencies_interval_2                   0
delinq_sum_interval_2                      0
active_loans_interval_3                   22
sum_loans_interval_3             1.20224e+06
delinquencies_interval_3                   0
delinq_sum_interval_3                      0
active_loans_interval_4                   22
sum_loans_interval_4             1.14428e+06
delinquenc

## Aggregating data per user

In [49]:
%%time
df2 = df.groupby('user_id').apply(feature_generator)

Wall time: 33.5 s


In [53]:
df2.drop(columns=['user_id']).to_csv('clean_data.csv')

## Creating dummies

In [21]:
df2.loc[:, df2.dtypes=='object']

,preferred_institution,preferred_account_type,preferred_credit_type,preferred_payment_freq
user_id,,,,
0,TIENDAS,Pagos Fijos,Línea de Crédito,Mensual
1,TIENDAS,Pagos Fijos,Préstamo Personal,Semanal
2,TIENDAS,Crédito Prendario,Préstamo Personal,Semanal
3,BANCO,Pagos Fijos,Préstamo de Nomina,Quincenal
4,BANCO,Revolvente,Tarjeta de Crédito,Mensual
...,...,...,...,...
995,FINANCIERA,Pagos Fijos,Préstamo Personal,Mensual
996,FINANCIERA,Pagos Fijos,Préstamo Personal,Mensual
997,BANCO,Pagos Fijos,Préstamo Personal,Semanal


In [26]:
df2 = df2.join(dummies.astype(int))

In [35]:
df2.iloc[:, -70:-49]

,sum_loans_interval_4,delinquencies_interval_4,delinq_sum_interval_4,active_loans_interval_5,sum_loans_interval_5,delinquencies_interval_5,delinq_sum_interval_5,active_loans_interval_6,sum_loans_interval_6,delinquencies_interval_6,...,active_loans_interval_7,sum_loans_interval_7,delinquencies_interval_7,delinq_sum_interval_7,tenure,longest_duration,preferred_institution,preferred_account_type,preferred_credit_type,preferred_payment_freq
user_id,,,,,,,,,,,,,,,,,,,,,
0,1144278.0,5,532712.50,20,426753.0,0,0.000,4,0.0,2,...,8,0.0,0,0.000000e+00,2490 days,1000.0,TIENDAS,Pagos Fijos,Línea de Crédito,Mensual
1,638283.0,0,0.00,8,638283.0,0,0.000,8,638283.0,0,...,8,638283.0,3,1.572048e+07,5037 days,360.0,TIENDAS,Pagos Fijos,Préstamo Personal,Semanal
2,20304.0,2,6566.00,0,0.0,0,0.000,1,0.0,0,...,2,0.0,0,0.000000e+00,1305 days,80.0,TIENDAS,Crédito Prendario,Préstamo Personal,Semanal
3,29640.0,0,0.00,2,29640.0,0,0.000,2,29640.0,0,...,2,29640.0,0,0.000000e+00,2517 days,72.0,BANCO,Pagos Fijos,Préstamo de Nomina,Quincenal
4,0.0,0,0.00,0,0.0,0,0.000,1,0.0,0,...,6,0.0,0,0.000000e+00,2132 days,0.0,BANCO,Revolvente,Tarjeta de Crédito,Mensual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,44378.0,1,69000.00,3,21501.0,1,8766.000,3,21501.0,0,...,4,21501.0,0,0.000000e+00,4670 days,104.0,FINANCIERA,Pagos Fijos,Préstamo Personal,Mensual
996,118260.0,2,152439.25,16,114163.0,2,322015.875,12,114163.0,2,...,13,21297.0,5,2.708841e+05,4801 days,60.0,FINANCIERA,Pagos Fijos,Préstamo Personal,Mensual
997,182741.0,2,35420.00,12,169958.0,2,37569.000,12,31802.0,0,...,21,14074.0,5,1.003753e+05,1913 days,80.0,BANCO,Pagos Fijos,Préstamo Personal,Semanal
